imports

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.utils import shuffle

In [10]:
def get_df(a,project):
    # load the model
    df = pd.read_csv(f'../data/{project}_final.csv')
    # drop all rows in df
    df = df.drop(df.index)
    print(df.shape)
    for p,u in a.items():
        if p == project:
            continue
        df1 = pd.read_csv(f'../data/{p}_final.csv')
        df = pd.concat([df,df1])
    # "access_modifier","return_type","method_name","parameters","returns","comments",content,path
    df = df.drop(columns=["access_modifier","return_type","method_name","parameters","returns","comments"],axis=1)
    # Count the number of rows in the "not_smell" class
    # not_smell_count = (df['catagory'] == 'not_smell').sum()

    # # Calculate the number of rows to drop (one-third of the count)
    # rows_to_drop = not_smell_count // 3

    # # Filter the DataFrame to exclude one-third of the "not_smell" class
    # filtered_smell = df[df['catagory'] == 'not_smeel'].sample(frac=(1/3))  # Selecting 2/3 of 'not_smell' instances
    # # print(filtered_smell.shape)
    
    # defalut_smell = df[df['catagory'] != 'not_smeel']
    # print(defalut_smell.shape)
    # df = pd.concat([filtered_smell, defalut_smell])
    
    train_df = shuffle(df)
    test_df = pd.read_csv(f'../data/{project}_final.csv')
    test_df = test_df.drop(columns=["access_modifier","return_type","method_name","parameters","returns","comments"],axis=1)
    # print(df['catagory'].value_counts())
    
    # df = df.sample(frac=1).reset_index(drop=True)
    
    # np.random.seed(42)
    # p = 0.1 # 10% for test set
    # prop = 1-p
    # temp_df = df.copy()
    # msk = np.random.rand(len(temp_df)) < prop
    # train_df = temp_df[msk]
    # test_df = temp_df[~msk]
    # print(train_df["label"].value_counts())
    # print(test_df["label"].value_counts())
    # test_df['index'] = test_df.index
    # train_df['index'] = train_df.index  
    return train_df, test_df

In [17]:
def get_df_temp(a,project):
    df = pd.read_csv(f'../data/{project}_final.csv')
    df.drop(columns=["access_modifier","return_type","method_name","parameters","returns","comments"],axis=1,inplace=True)
    train_df, test_df = train_test_split(df, train_size=.8, random_state=42)
    print(train_df.shape)
    print(test_df.shape)
    return train_df, test_df

In [14]:
# Create the KNN model
def model_(train_df,test_df):
    tfidf_vectorizer = TfidfVectorizer(max_features=50)
    X_train = tfidf_vectorizer.fit_transform(train_df['content'])
    X_test = tfidf_vectorizer.transform(test_df['content'])
    train_df.drop(columns=['content'], inplace=True)
    test_df.drop(columns=['content'], inplace=True)
    y_train = train_df.values
    y_test = test_df.values
    # Create the MultiOutputClassifier
    log_reg = LogisticRegression(max_iter=1000) 
    multi_target_log_reg = MultiOutputClassifier(log_reg, n_jobs=-1)

    # Train the model
    multi_target_log_reg.fit(X_train, y_train)

    # Make predictions
    predictions = multi_target_log_reg.predict(X_test)

    print(classification_report(y_test, predictions, target_names=['get','DOS','is','set']))

In [12]:
a = {
    "buildship": "https://github.com/eclipse/steady.git",
    "eclips-collections": "https://github.com/eclipse/eclipse-collections.git",
    "hawkbit": "https://github.com/eclipse/hawkbit.git",
    "jkube": "https://github.com/eclipse/jkube.git",
    "kura": "https://github.com/eclipse/kura.git",
    "jifa": "https://github.com/eclipse/microprofile.git",
    "milo": "https://github.com/eclipse/milo.git",
    "openvsx": "https://github.com/eclipse/openvsx.git",
    "steady": "https://github.com/eclipse/steady.git",
    "xtext": "https://github.com/eclipse/xtext.git"
}

In [20]:
for project, url in a.items():
    train_df, test_df = get_df_temp(a,project)
    model_(train_df,test_df)

(3904, 5)
(977, 5)
              precision    recall  f1-score   support

         get       0.00      0.00      0.00        31
         DOS       0.98      0.90      0.94       146
          is       0.50      0.40      0.44       257
         set       0.60      0.24      0.34       114

   micro avg       0.67      0.48      0.56       548
   macro avg       0.52      0.38      0.43       548
weighted avg       0.62      0.48      0.53       548
 samples avg       0.26      0.26      0.26       548



/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

(5976, 5)
(1495, 5)
              precision    recall  f1-score   support

         get       0.78      0.22      0.35       175
         DOS       0.00      0.00      0.00        33
          is       0.83      0.61      0.70       206
         set       0.82      0.49      0.61       468

   micro avg       0.82      0.45      0.58       882
   macro avg       0.61      0.33      0.42       882
weighted avg       0.78      0.45      0.56       882
 samples avg       0.26      0.26      0.26       882



/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

(2543, 5)
(636, 5)
              precision    recall  f1-score   support

         get       0.80      0.20      0.32        59
         DOS       0.92      0.45      0.61        51
          is       0.48      0.29      0.36       141
         set       0.66      0.50      0.57       157

   micro avg       0.64      0.38      0.48       408
   macro avg       0.72      0.36      0.47       408
weighted avg       0.65      0.38      0.47       408
 samples avg       0.24      0.23      0.23       408

(2481, 5)
(621, 5)


...

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

              precision    recall  f1-score   support

         get       0.79      0.47      0.59        49
         DOS       0.88      0.92      0.90       201
          is       0.47      0.38      0.42       650
         set       0.60      0.39      0.47       201

   micro avg       0.60      0.49      0.54      1101
   macro avg       0.69      0.54      0.60      1101
weighted avg       0.58      0.49      0.53      1101
 samples avg       0.26      0.26      0.26      1101

(2436, 5)
(610, 5)
              precision    recall  f1-score   support

         get       0.96      0.69      0.80        77
         DOS       1.00      0.98      0.99        54
          is       0.31      0.11      0.17        97
         set       0.43      0.12      0.19        84

   micro avg       0.76      0.41      0.53       312
   macro avg       0.68      0.48      0.54       312
weighted avg       0.62      0.41      0.47       312
 samples avg       0.21      0.21      0.21       312

(44

/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         get       0.56      0.21      0.31       104
         DOS       0.94      0.82      0.87       144
          is       0.50      0.37      0.43       265
         set       0.62      0.18      0.28       113

   micro avg       0.65      0.41      0.51       626
   macro avg       0.66      0.40      0.47       626
weighted avg       0.63      0.41      0.48       626
 samples avg       0.23      0.23      0.23       626

(72731, 5)
(18183, 5)
              precision    recall  f1-score   support

         get       0.82      0.72      0.77      3297
         DOS       0.76      0.37      0.50       793
          is       0.56      0.04      0.08      3545
         set       0.59      0.15      0.24      2421

   micro avg       0.77      0.32      0.45     10056
   macro avg       0.68      0.32      0.40     10056
weighted avg       0.67      0.32      0.38     10056
 samples avg       0.17      0.17      0.17     10056



/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
